# Daten: Einlesen, Bereinigen & Visualisieren

## Import von Bibliotheken  
Wir importieren die Bibliotheken: 
 - glob bietet Funktionen, um Pfadnamen im Unix-Stil über Muster zu erweitern. Dies ist sehr Hilfreich, wenn wir mit vielen Dateien und Ordnern arbeiten. 
 - pandas ist eine sehr mächtige Bibliothek für inhomogene tabellarische Daten sog. DataFrames. Sie ist ebenfalls eine der de facto Standardbibliotheken im Bereich Data Science und Machine Learning. Hier verwenden wir sie für die Bearbeitung der Daten in einer .csv bzw. .xlsx Datei. 
 - openpyxl erlaubt das Öffen und Bearbeiten von Excel 2010 Dateiformaten. Ausserdem bietet sie eine Schnittstelle zu pandas. Prinzipiell ist openpyxl die bessere Wahl, wenn die Struktur/Formatierung der Excel-Datei wichtig ist. Wenn es rein um die enthaltenen Daten und deren Verarbeitung geht, ist pandas meist die bessere Wahl (und ggf. einer Konvertierung des Dateiformats)

In [ ]:
import pandas as pd
import seaborn as sns
import glob
import openpyxl
import os
import matplotlib
%matplotlib notebook
# mit der notebook option fuer matplotlib bekommen wir interaktive plots, welche sich besser zur exploration eignen
# fuer reports sollten die grafiken statisch erstellt und angepasst werden mit %matplotlib inline

# dieser befehl laesst uns die groesse der plots global veraendern
#matplotlib.rcParams['figure.figsize'] = [16,9]
import matplotlib.pyplot as plt

## Einlesen einer CSV-Datei aus einem lokalen Ordner

pandas bietet uns Einlese-Funktionen für verschiedene Formate, in diesem Beispiel erwartet das Notebook einen Datensatz von FIFA Spielern im csv format im Ordner `resources/data`.

In [ ]:
df = pd.read_csv('../resources/data/fifa19.csv')

Während der Datenbearbeitung will man oft die Struktur der Daten vor und nach einzelnen Operationen überprüfen, um sicherzugehen, dass die Operationen korrekt ausgeführt wurden bzw. welche zunächst anzuwenden sind. Die Funktion head() gibt die ersten 5 Zeilen (konfigurierbar) eines DataFrame aus.
[pandas.DataFrame.head() Doku](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)

In [ ]:
df.head()

Rohdaten einsehen ist jedoch nicht immer hilfreich oder ueberschaubar (df.head() z.B. druckt nicht alle Spalten im Notebook), weswegen ein DataFrame Methoden zur Uebersicht bereitstellt. 
- [pandas.DataFrame.describe() Doku](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
- [pandas.DataFrame.info() Doku](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

In [ ]:
df.info()

In [ ]:
df.describe()

Die Datei enthaelt also Informationen ueber einzelene Spieler, wie deren Spielstatistiken, aber auch Nationalität und Club-Zugehörigkeit. Die Art der Daten ist gemischt, die statistischen Daten sind Dezimalzahlen, die Clubzugehörigkeit oder Nationalität sind nominelle Variablen (Eine aus N Möglichkeiten, nicht numerisch/geordnet).  
Schauen wir uns zuerst einmal an, wieviele Clubs überhaupt in dem Datensatz gelistet sind. Dazu gibt es einen einfachen Trick: Man berechnet die der einzigartigen Einträge (.unique()) der Spalte ("Club") und berechnet die Länge des Resultats.

Welche anderen Eigenschaften des Datensatzes könnten wir zusaetzlich untersuchen?

Wie gehen wir mit fehlenden Informationen um?

## Erster Schritt: Exploration und Visualisierung der Daten
Am Anfang eines datengestuetzten Projektes steht die Exploration und Visualisierung der Daten. Eine moegliche Fragestellung in diesem Kontext ist es, mehr ueber die Leistungsfaehigkeit einzelner Clubs zu erfahren. Dazu koennen wir uns einzelne Variablen und Kombinationen von Spielervariablen ansehen und daraus Rueckschluesse ziehen. Ein erster guter Ansatz hierbei besteht darin, statistische Kennzahlen unterschiedlicher Variablen auszuwerten.  

Dieser Datensatz enthaelt bereits mehrere Leistungskennzahlen fuer jeden Spieler - wir koennten also zum Beispiel die mittleren Leistungswerte der Spieler pro Club ansehen. Da es sehr viele Clubs gibt, untersuchen wir zuerst nur die jeweils drei "schlechtesten" und "besten" Clubs (nach durchschnittlicher 'Overall'-Spielerbewertung, nicht nacht wirklichen Ergebnissen!) und visualisieren die Verteilung der Kennzahlen in einem sogenannten 'boxplot'.  
Ein boxplot ist sozusagen die 'Draufsicht' auf eine statistische Verteilung, bei der Mittelwert, Schiefe durch 'Boxen' (enhalten die mittleren 50% der Daten) und 'Antennen' ausgedrueckt werden.


### 1. Boxplot der Spalte "Overall" für die drei besten bzw. schlechtesten Clubs


Zuersteinmal muessen wir die "besten/schlechtesten" Clubs herausfiltern. Das machen wir, indem wir unseren DataFrame nach der Spalte der Clubzuordnung gruppieren. Die Funktion dazu heisst naheliegend "groupby()" [pandas.DatraFrame.groupby() Doku](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html).  
Die so herausgefundenen Clubnamen kombinieren wir in einer Liste und indizieren anhand dieser dann den DataFrame:

Nun haben wir die Selektion der 6 Clubs in dem dataframe 'df_selection' und koennen darueber einen boxplot erstellen:

Bei Inter Mailand, Juventus und Limerick gibt es offensichtlich einige Ausreißer, die wir uns gesondert noch anschauen moechten.

### 2. Bearbeiten und Visualiseren der Marktwerte (Spalte 'Release Clause') 

Eine weitere Moeglichkeit, die uns dieser Datensatz bietet, ist eine Analyse der Marktwerte der aufgezeichneten Spieler.
Dabei ergibt sich in der  Spalte 'Release Clause' das Problem, dass der Wert im Textformat vorliegt und somit nicht direkt zur Berechnung herangezogen werden kann. Hierzu muessen wir diese Spalte in einen numerischen Datentyp umwandeln. Diese Information beziehen wir durch die Auflistung der Datentypen aus df.info() bzw. df.dtypes.

Zuerst muessen wir herausfinden, welche Kodierungen fuer Waehrungen und SI Einheiten ueberhaupt in der Spalte 'Release Clause'  vorkommen. Eine pythonische Moeglichkeit hierzu: wir fuehren alle einzelnen Werte in einen grossen String zusammen und nutzen den Set-Trick aus der Einfuehrung um die einzigartigen Werte zu extrahieren:

### 3. Untersuchung der physischen Eigenschaften einzelner Spieler nach ihrer Position
In einem weiteren Scatterplot moechten wir uns nun Staerke vs. Ausdauer der Spieler, coloriert nach Position, genauer ansehen.
Auch hierzu sind ein paar Anpassungen am Datensatz notwendig, um diese Grafik darstellen zu koennen.
Wir beginnen mit der Untersuchung bei der Variable 'Position'.

Dies ist zwar eine sehr genaue Auflistung, jedoch eventuell etwas zu kleinteilig um diese Variable direkt in entsprechenden Grafiken zu nutzen. Deshalb konvertieren wir die Positionen der Spieler in die Werte  Angriff, Mittelfeld, Verteidigung und Torwart, um visuelle Klarheit zu schaffen.

Oft bestaetigen die Daten, was wir sowieso schon wissen oder vermuten - in diesem Fall sehen wir recht gut, dass Torwarte tendenziell weniger Ausdauer als Feldspieler haben.

### 4. Weitere Plots!

Was interessiert uns noch? Um weitere Inspiration zu erhalten, koennten wir uns die Dokumentation von [seaborn](https://seaborn.pydata.org/), unserer Visualisierungsbibliothek, genauer ansehen. 

## Exkurs: Gruppieren und Export der Daten in ein anderes Format 
Als naechstes waere es interessant, die Spieler nach ihrem Club zu gruppieren. Dazu gibt es in Pandas die Funktion groupby(), welche den DataFrame anhand von Kriterien aufteilt, in diesem Fall nehmen wir als Kriterium die Variable "Club".  
Das Ergebnis der Operation ist eine iterierbare Sammlung von Gruppierungen, für die wir je Gruppe den Namen (des Clubs) und die dazugehörigen Daten erhalten.  
Zu Demonstrationszwecken teilen wir diese Daten auf jeweils eine eigene Excel-Datei auf (nur um zu zeigen, wie man mit Excel-Dateien arbeitet).  
Wir lassen also eine for-Schleife über die GroupBy-Sequenz laufen und versuchen, für jede Gruppe eine Datei zu erstellen. Da diese Operation fehlschlagen kann, sollten wir sie mit einem try-except Block abfangen, welcher explizit angibt, was im Fehlerfall zu tun ist:  
[Exceptions Docs](https://docs.python.org/3/library/exceptions.html)  

Um die neuen Dateien von der Quelldatei zu trennen, erstellen wir erst noch einen eigenen Unterordner. Wenn Sie die Zelle mehr als einmal Ausführen wird es einen Fehler geben, da der Ordner ja schon vorhanden ist und nicht überschrieben wird.

In [ ]:
path = "../resources/data/clubs/"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Was passiert hier? Einer der Clubnamen enthaelt einen Forward-Slash ('/'), welcher im Pfadnamen als Unterordner interpretiert wird, welcher nicht exitiert. Wir müssen den String also erst bereinigen, bevor wir diese Datei schreiben können.  
Dazu waehlen wir die gesamte Spalte "Club" an und führen einen Textersatz durch. Die Operation wird dann auf alle Elemente der Spalte angewendet. Mit der erneuten Zuweisung wird dann das Original überschrieben.

Der neue Clubname lautet also "FK Bodø Glimt". Im gesamten DataFrame können wir die Adressierung nun testen und uns ansehen, welche Spieler dort unter Vertrag sind. In pandas gibt es mehrere Moeglichkeiten, Spalten anzuwaehlen. Die gaengigste ist jedoch das sog. Boolean Indexing, also eine Anwahl über einen logischen Ausdruck. In pandas duerfen diese Ausdrücke direkt in die eckigen Klammern der Adressierung geschrieben werden.  
Das koennte sich so lesen: Waehle Eintraege aus 'df', für die die Spalte 'Club' gleich der Zeichenkette 'FK Bodø Glimt' ist.

Nachdem diese Operation das gewuenschte Ergebnis erbracht hat, koennen wir diese Datei nachtraeglich ebenfalls exportieren:

## Einlesen von mehreren lokalen Dateien via Pattern Matching  
Nachdem wir nun eine Menge neuer Dateien erzeugt haben, wollen wir diese alle automatisiert einlesen. Fuer Sie koennten das z.b. Datenbank-Exporte aus eienm ERP-System oder eine Sammlung von Logfiles sein. Mit der glob-Bibliothek finden wir nun alle vorhandenen Dateien/Pfade, welche eine xlsx-Datei mit beliebigem Namen (gekennzeichnet durch die "Wildcard" *) sind. Das Resultat sind alle gefundenen Dateinamen unter dem gegebenen Muster: 

Achtung! Die Anzahl der gefundenen Dateien ist um 1 kleiner, als die Anzahl der Clubs im Ausgangsdatensatz!  
### Was ist hier passiert?  
Wir sollten herausfinden, worin die Diskrepanz besteht. Dazu benutzen wir wieder Mengenoperationen. 
Zuerst brauchen wir die Clubnamen der erzeugten Dateien - dazu teilen wir die matched-Pfade zwischen Dateiname und Dateiendung mit der Split-Funktion (Die jeweiligen Abgespaltenen Teile können mit eckigen Klammern hinter der Funktion adressiert werden). Dieses Ergebnis wandeln wir in ein Set um (Reminder: Sets enthalten keine Duplikate)

Beim Original DataFrame ist es einfach, dort koennen wir wieder die unique() Funktion nutzen und muessen ebenfalls ein set erstellen um die Mengenoperatoren nutzen zu können:

Nun schauen wir, welche Eintraege entweder in der einen, oder der anderen Menge enthalten sind:

Der Eintrag nan ('not a number', bzw. NA in neueren pandas Versionen) signalisiert fehlende oder invalide Werte. Das heißt in unserem Datensatz gab es auch Spieler ohne Club-Zuordnung also einem leeren Textfeld.  
Diese muessen wir also noch gesondert behandeln, wir nennen sie 'Free Agent'. Wir waehlen also die Zeilen aus, in denen die Spalte Club den wert nan enthaelt und ueberschreiben ihn mit 'Free Agent' mit der DataFrame Funktion loc() fuehren wir diese Operation "in-place" aus (also ohne zusaetzlichen Speicher zu verbrauchen).

Kurze Verifikation der Prozedur:

Und auch hier muessen wir diesen Fall wieder einzeln nachtraeglich bearbeiten, da Dateien mit nan Namen nicht erzeugt wurden.

Erneutes Matching mittels glob ergibt nun die korrekte Laenge:

Um bestimmte Einträge z.B. Spieler in unseren Dateien zu finden, kombinieren wir nun alles hier Gelernte in einem Code-Block.  
 - Iterieren von matched file names mittels glob  
 - Lesen der Excel-Files mit pandas  
 - Addressierung der Eintraege über pandas Funktionalität  
 - Export der bearbeiteten Daten durch pandas

Wieder koennen wir das Boolean Indexing verwenden um auf Namen zu pruefen. Da wir nicht genau wissen, ob mehrere Spieler den gleichen Namen tragen, fragen wir nur ab, ob der Name einen gewissen String enthaelt und kombinieren dies mit der Nationalitaet um die Suche einzugrenzen.  
Ist ein Spieler gefunden, erhoehen wir in diesem Beispiel seine "Release Clause". Dies geht natuerlich für beliebige Eintraege und haengt vom Anwendungsfall ab. 